# 편각 계산하기

## 요약

지구 자기장 모델을 만들어 편각을 계산하기 위해 두 가지 시도를 했다.

첫째, 코사인의 법칙을 구면삼각형으로 확장하여 이를 이용하였다. 지구를 완벽한 구로 생각하고 관찰자, 진북, 자북을 잇는 구면 삼각형을 그린다. 여기서 관찰자 쪽에 생기는 각을 계산하였다.

둘째, 자북과 자남 근처에 각각 자기 홀극이 존재한다고 가정하고 자기장 벡터를 계산하였다. 북쪽에는 S극, 남쪽에는 N극을 맨틀과 외핵 경계면 깊이에 두었다. 자기장의 세기는 거리 제곱에 반비례한다고 가정하였다. 지표의 특정 위치에서 자기장의 수평 성분을 계산하고 벡터 연산을 통해 편각을 구했다.

각 방법으로 지구 자기 편각 지도를 그려본다. WMM(World Magnetic Model)과 비교해보고 차이를 분석한다. 

## 방법 1: 구면 코사인 법칙

### 코사인 법칙 구면삼각형으로 확장하기

반지름이 1인 구 위에 구면삼각형 ABC를 잡자. 점 A, B, C의 대변을 각각 a, b, c라 하자. 그러면 다음 식이 성립한다. (A, B, C에서 $\angle$ 기호는 생략하였다.)

$$
\cos a = \sin b \sin c \cos A + \cos b \cos c \\[0.5em]
\cos A = {\cos a - \cos b \cos c \over \sin b \sin c}
$$

따라서 구면삼각형의 세 변의 길이를 알면 세 각도 알 수 있다.

### 편각 계산하기

지구를 완벽한 구로 가정하고 진북, 자북, 관찰자를 세 점으로 하는 구면삼각형을 그릴 수 있다.
세 점의 위도, 경도가 주어지면 각 변의 길이를 계산하고 이를 이용하여 편각을 구한다.

진북을 점 A, 자북을 점 B, 관찰자를 점 C라 하자. 
다음 식을 통해 각 변의 길이와 편각을 구할 수 있다.
$\phi_m$, $\lambda_m$ 은 자북의 위도, 경도이고 $\phi_o$, $\lambda_o$는 관찰자의 위도, 경도이다.

$$
A = | \lambda_m - \lambda_o | \\[0.5em]
b = {\pi \over 2} - \phi_o \\[0.5em]
c = {\pi \over 2} - \phi_m \\[0.5em]
\cos a = \sin b \sin c \cos A + \cos b \cos c \\[0.7em]
\sin a = \sqrt{1 - \cos^2 a} \\[0.5em]
\cos C = {\cos c - \cos a \cos b \over \sin a \sin b}
$$

$\cos a$를 구할 때, $\cos C$를 구할 때 각각 코사인 법칙이 사용되었다.

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Magnetic pole location was from WMM pole locations (year 2023)
# https://www.ngdc.noaa.gov/geomag/data/poles/WMM2020_NP.xy
magnetic_north_lon = 146.826
magnetic_north_lat = 86.146
magnetic_south_lon = 135.417
magnetic_south_lat = -63.930

feature_x = np.arange(-180, 180, 0.2)
feature_y = np.arange(-90, 90, 0.2)

# Observer longitude and latitude
lon, lat = np.meshgrid(feature_x, feature_y)

sign = np.sign((lon - magnetic_north_lon) % 360 - 180)
A = np.deg2rad(np.abs(magnetic_north_lon - lon))
b = np.deg2rad(90 - lat)
c = np.deg2rad(90 - magnetic_north_lat)
cos_a = np.sin(b) * np.sin(c) * np.cos(A) + np.cos(b) * np.cos(c)
sin_a = np.sqrt(1 - np.square(cos_a))
cos_C = np.clip((np.cos(c) - cos_a * np.cos(b)) / (sin_a * np.sin(b)), -1, 1)

# Magnetic declination
Z = np.rad2deg(np.arccos(cos_C)) * sign

In [ ]:
fig = go.Figure(
    data=[
        go.Contour(
            x=feature_x,
            y=feature_y,
            z=Z,
            contours=dict(
                coloring="heatmap",
                showlabels=True,  # show labels on contours
                labelfont=dict(  # label font properties
                    size=12,
                    color="white",
                ),
            ),
            showlegend=False,
            ncontours=20,
        ),
    ]
)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.show()
# fig.write_image("fig1.png", scale=2)

<!-- Resized, cropped for print
<div style="width: auto; height:260px; overflow:hidden;"><img src="./fig1.png" style="margin-top: -65px;height: 375px;" height="300px"></div>
-->

<img src="./fig1.png" width="75%">

### 결과 분석

거의 모든 지역에서 편각이 0에 가깝고 극지방에서만 급격히 변하는 것을 확인할 수 있다. 또한 실제 지구와 다르게 매우 대칭적이다.

오류의 원인으로 다음과 같은 문제점이 있다고 생각한다.

 - 지구 자기장의 수평 성분이 구면에서 자북의 방향과 일치하지 않을 수 있다.
 - 지구 자기장은 실제로 대칭적이지 않다. 자북과 자남이 정반대가 아니다.
 - 지구는 완벽한 구가 아니다.
 

## 방법 2: 자기장 벡터 계산

### 편각 계산하기


지구 자기장이 두 개의 자기 홀극에 의해 만들어진다고 가정한다. 자북극 아래에 S극을 놓고, 자남극 아래에 N극을 놓는다. 자기 홀극의 깊이는 맨틀-외핵 경계면 부근으로 설정하였다.

지구를 중심이 원점이고 반지름이 1인 구로 가정한다. 자전축이 z축이 되도록 좌표를 설정한다.
자북의 S극의 위치벡터를 $\overrightarrow{p_n}$, 자남의 N극의 위치벡터를 $\overrightarrow{p_s}$라 하자.

구면 위의 관찰자의 위치벡터를 $\overrightarrow{x}$라 하자.
$\overrightarrow{x}$ 에서 자기장 벡터는 다음과 같다.

$$
\overrightarrow{B} = 
      { \overrightarrow{p_n} - \overrightarrow{x} \over 
\left\| \overrightarrow{p_n} - \overrightarrow{x} \right\| ^3 } - 
      { \overrightarrow{p_s} - \overrightarrow{x} \over 
\left\| \overrightarrow{p_s} - \overrightarrow{x} \right\| ^3 }
$$

자기장 방향만 계산하면 되기 때문에 상댓값으로 계산한다.

$\overrightarrow{x}$에서 구면에 대한 법선벡터는 $\overrightarrow{x}$이다.
자기장의 수평 방향은 $\overrightarrow{x}$에서 구면의 접평면에 자기장 벡터를 사영한 것이다.
자기장의 수직 성분을 $-k \overrightarrow{x}$라 하면
수평 성분은 $\overrightarrow{B} + k \overrightarrow{x}$이다.
따라서 $k$는 다음 식을 만족한다.

$$
( \overrightarrow{B} + k \overrightarrow{x} ) \cdot \overrightarrow{x} = 0
$$

수평 성분을 $\overrightarrow{d_m}$이라 하자.
$$
\overrightarrow{d_m} = \overrightarrow{B} + k \overrightarrow{x}
$$

식을 정리하여 $k$를 구할 수 있다. 구의 반지름이 1이기 때문에 $\overrightarrow{x} \cdot \overrightarrow{x} = 1$이다.

$$
\overrightarrow{B} \cdot \overrightarrow{x}
+ k \overrightarrow{x} \cdot \overrightarrow{x} = 0
\\
k = -\overrightarrow{B} \cdot \overrightarrow{x}
$$

진북의 위치벡터를 $\overrightarrow{x_t}$라 하자.
그러면 $\overrightarrow{x}$에서 진북으로의 방향 벡터는 $\overrightarrow{x_t} - \overrightarrow{x}$이다.

방향 벡터의 수직 성분을 $-k' \overrightarrow{x}$라 하면 수평 성분은 $(\overrightarrow{x_t} - \overrightarrow{x}) + k' \overrightarrow{x}$이다.
$k'$는 다음 식을 만족한다.

$$
( (\overrightarrow{x_t} - \overrightarrow{x}) + k' \overrightarrow{x} ) \cdot \overrightarrow{x} = 0
\\
$$

$$
\begin{align*}
k' &= - (\overrightarrow{x_t} - \overrightarrow{x}) \cdot \overrightarrow{x} \\
   &= \; 1 - \overrightarrow{x_t} \cdot \overrightarrow{x}
\end{align*}
$$

수평 성분을 $\overrightarrow{d_t}$라 하자. 이를 계산하면 다음과 같다.
$$
\begin{align*}
\overrightarrow{d_t}
&= (\overrightarrow{x_t} - \overrightarrow{x}) + k' \overrightarrow{x} 
\\
&= \overrightarrow{x_t} - (\overrightarrow{x_t} \cdot \overrightarrow{x}) \overrightarrow{x}
\end{align*}
$$

$\overrightarrow{x}$에서 편각을 $\theta$라 하자.
$\theta$가 $\overrightarrow{d_m}$와 $\overrightarrow{d_t}$가 이루는 각이기 때문에 내적, 외적을 이용하여 이를 계산할 수 있다.

$$
\cos \theta = 
{ \overrightarrow{d_m} \cdot
\overrightarrow{d_t}
\over \left\| \overrightarrow{d_m} \right\|
\left\|\overrightarrow{d_t} \right\|}
\\[1em]
\sin \theta = { \overrightarrow{d_m} \times
\overrightarrow{d_t}
\over \left\| \overrightarrow{d_m} \right\|
\left\|\overrightarrow{d_t} \right\|}
\cdot \overrightarrow{x}
$$

In [ ]:
def gcs_to_vec(lat, lon):
    x = np.cos(lat) * np.sin(lon)
    y = np.cos(lat) * np.cos(lon)
    z = np.sin(lat)
    return np.array([x, y, z])


pn_depth = 3600 / 6400
ps_depth = 3600 / 6400


In [ ]:
# Observer longitude and latitude
lon, lat = np.meshgrid(feature_x, feature_y)

x = gcs_to_vec(np.deg2rad(lat), np.deg2rad(lon))

pn = gcs_to_vec(np.deg2rad(magnetic_north_lat), np.deg2rad(magnetic_north_lon))
pn = pn * pn_depth
pn = np.broadcast_to(np.reshape(pn, (3, 1, 1)), np.shape(x))
ps = gcs_to_vec(np.deg2rad(magnetic_south_lat), np.deg2rad(magnetic_south_lon))
ps = ps * ps_depth
ps = np.broadcast_to(np.reshape(ps, (3, 1, 1)), np.shape(x))

x_t = np.broadcast_to(np.reshape([0, 0, 1], (3, 1, 1)), np.shape(x))

pn_min_x = pn - x
pn_min_x_len_cubed = np.power(np.sum(np.square(pn_min_x), axis=0), 1.5)

ps_min_x = ps - x
ps_min_x_len_cubed = np.power(np.sum(np.square(ps_min_x), axis=0), 1.5)

B = pn_min_x / pn_min_x_len_cubed - ps_min_x / ps_min_x_len_cubed
k = -np.sum(B * x, axis=0)
d_m = B + k * x
# Normalize d_m
d_m = d_m / np.sqrt(np.sum(np.square(d_m), axis=0))

# Since x_t is [0, 0, 1], x_t dot x is the z component of x.
d_t = x_t - x[2] * x
# Normalize d_t
d_t = d_t / np.sqrt(np.sum(np.square(d_t), axis=0))

cos_theta = np.clip(np.sum(d_t * d_m, axis=0), -1, 1)
sin_theta = np.clip(np.sum(x * np.cross(d_t, d_m, 0, 0, 0), axis=0), -1, 1)

# Magnetic declination
theta = np.rad2deg(np.arccos(cos_theta)) * np.sign(sin_theta)

In [ ]:
fig = go.Figure(
    data=[
        go.Contour(
            x=feature_x,
            y=feature_y,
            z=theta,
            contours=dict(
                coloring="heatmap",
                showlabels=True,  # show labels on contours
                labelfont=dict(  # label font properties
                    size=12,
                    color="white",
                ),
            ),
            showlegend=False,
            ncontours=20,
        ),
    ]
)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.show()
# fig.write_image("fig2.png", scale=2)

<!-- Resized, cropped for print
<div style="width: auto; height:260px; overflow:hidden;"><img src="./fig2.png" style="margin-top: -65px;height: 375px;" height="300px"></div>
-->

<img src="./fig2.png" width="75%">

### 결과 분석

여전히 WMM에 비하면 단순하지만 방법 1보다는 더 비슷해졌다. 대체로 남반구에서 편각의 크기가 큰데 이는 자남극이 극에서 더 멀기 때문이라고 생각한다. 여기서는 자북과 자남 부근에 자기 홀극 각각 1개씩을 놓아 더 복잡한 형태를 만들 수 없다. 지구 곳곳에서 자기장 벡터를 측정하고 이를 바탕으로 여러 개의 자기 홀극으로 이루어진 모델을 만들면 실제 지구 자기장과 더 유사한 모델을 얻을 수 있을 것이라 생각한다.